In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Real Data

In [2]:
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle

def load_dataset_test(full_path = '/content/drive/MyDrive/Privacy/real_data_test.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  print(data)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_test, y_test = load_dataset_test()
X_test = expit(X_test)
# print(X.shape,y.shape)

            0         1        2        3  4
63    1.79390  -1.11740  1.54540 -0.26079  0
529  -1.38850  12.50260  0.69118 -7.54870  0
258   3.74400   0.79459  0.95851  1.00770  0
589   0.93611   8.64130 -1.63510 -1.30430  0
141   1.73170  -0.34765  4.19050 -0.99138  0
...       ...       ...      ...      ... ..
312   1.59020   2.29480  3.24030  0.18404  0
491   3.06320  -3.33150  5.13050  0.82670  0
1067 -0.82601   2.96110 -1.28640 -1.46470  1
114   1.78190   6.91760 -1.27440 -1.57590  0
253   5.28680   3.25700 -1.37210  1.16680  0

[372 rows x 5 columns]


# Load Synthetic Data

In [3]:
import numpy as np
import pickle
import pandas as pd
from pandas import read_csv

with open('/content/drive/MyDrive/Privacy/ACPATE_results/X_ACPATE_NEW_low_epoch.csv', 'rb') as f:
    data = pickle.load(f)
with open('/content/drive/MyDrive/Privacy/ACPATE_results/Y_ACPATE_NEW_low_epoch.csv', 'rb') as f:
    labels = pickle.load(f)
print(data.shape)
print(labels.shape)

X_ACPATE = data
Y_ACPATE = labels

(1000, 4)
(1000,)


# Evaluation Metrics

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import tensorflow as tf

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model = Sequential([
Dense(input_dim = 4, units = 2, activation = 'relu'),
Dense(units = 4, activation = 'relu'),
# Dropout(0.2),
# Dense(units = 20, activation = 'relu'),
Dense(units = 10, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])


# Evaluate Private Data Generation Using a Neural Network

In [13]:
# evaluate the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', f1_m, precision_m, recall_m])
model.fit(X_ACPATE, Y_ACPATE, batch_size = 20, epochs =100)
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
#pred = model.predict(X_test)

print(loss, accuracy, f1_score, precision, recall)

Epoch 1/100
50/50 [==============================] - 1s 837us/step - loss: 0.3149 - accuracy: 0.9410 - f1_m: 0.9309 - precision_m: 0.9207 - recall_m: 0.9473
Epoch 2/100
50/50 [==============================] - 0s 837us/step - loss: 0.2750 - accuracy: 0.9477 - f1_m: 0.9393 - precision_m: 0.9273 - recall_m: 0.9600
Epoch 3/100
50/50 [==============================] - 0s 861us/step - loss: 0.2349 - accuracy: 0.9549 - f1_m: 0.9439 - precision_m: 0.9186 - recall_m: 0.9778
Epoch 4/100
50/50 [==============================] - 0s 963us/step - loss: 0.2133 - accuracy: 0.9584 - f1_m: 0.9535 - precision_m: 0.9448 - recall_m: 0.9658
Epoch 5/100
50/50 [==============================] - 0s 839us/step - loss: 0.2124 - accuracy: 0.9428 - f1_m: 0.9406 - precision_m: 0.9313 - recall_m: 0.9576
Epoch 6/100
50/50 [==============================] - 0s 930us/step - loss: 0.1696 - accuracy: 0.9593 - f1_m: 0.9553 - precision_m: 0.9309 - recall_m: 0.9853
Epoch 7/100
50/50 [==============================] - 0s 1m